**Project #1 - dashboard #2**

Using the plotly and dash library, create a dashboard consisting of one visualization:
1. map using the dark style from Mapbox, showing the markers of countries from the selected continent; with maker size depending on the value of the selected metric. For example, the user will be able to select the continent Europe and the measure: Total cases, resulting in markers (circles) appearing on the map in Europe. The larger the marker, the more cases in the country.
2. For calculations, use the data from the last day in the provided dataset.
3. The dashboard should consist of:
- title that changes depending on the value of the filters according to the pattern: COVID-19 - {metric} in {continent}.
- two filters:
 - filter to select the continent
 - filter to select a metric (total number of positive cases, total number of deaths, total number of tests, total number of vaccinations, total number of vaccinated people)
- titles (labels) of filters
- visualizations from the first point
4. When you hover over a marker, the name of the country appears in addition to the coordinates and metrics.
5. To run the dash application in Google Colab, use the solution presented in the course, based on the threading and google.colab libraries. Use the function that displays the application in a new browser tab.

In [1]:
# Install pandas, matplotlib, plotly, dash, and threading if necessary
!pip install pandas matplotlib plotly dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.3 MB/s eta 0:00:00


In [2]:
# Import the libraries with aliases
import dash
import pandas as pd
import numpy as np
import plotly.express as px
import threading
from dash import dcc, html, Input, Output
from google.colab import output, drive

drive.mount('/drive')

Mounted at /drive


In [3]:
df = pd.read_csv('/drive/My Drive/FINAL PROJECTS/Covid_19/project_1_python.csv')
df.tail(3)

,index,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
180474,190610,ZWE,Africa,Zimbabwe,2022-06-16,254502.0,115.0,5526.0,1.0,NaN,...,6274305.0,4570349.0,1029463.0,7126.0,15092171.0,19.6,1899.775,61.49,-20.0,30.0
180475,190611,ZWE,Africa,Zimbabwe,2022-06-17,254753.0,251.0,5533.0,7.0,NaN,...,6276402.0,4574222.0,1031790.0,8297.0,15092171.0,19.6,1899.775,61.49,-20.0,30.0
180476,190612,ZWE,Africa,Zimbabwe,2022-06-18,254753.0,NaN,5533.0,0.0,NaN,...,NaN,NaN,NaN,NaN,15092171.0,19.6,1899.775,61.49,-20.0,30.0


In [4]:
# Searching for the max date when the numbers were taken for the df (assuming for all the countries) that is why it will be more than enough to work with new df
# Also we need to fill NaN values with 0 so graph can be created
df_dash_2 = df[df['date'] == df['date'].max()].fillna(0)

df_dash_2.head(3)
# print(df_dash_2.to_string())

,index,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
845,845,AFG,Asia,Afghanistan,2022-06-18,181574.0,40.0,7713.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39835428.0,18.6,1803.987,64.83,33.0,65.0
1690,1690,ALB,Europe,Albania,2022-06-18,277141.0,0.0,3497.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2872934.0,38.0,11803.431,78.57,41.0,20.0
2535,2535,DZA,Africa,Algeria,2022-06-18,265971.0,3.0,6875.0,0.0,0.0,...,0.0,0.0,0.0,0.0,44616626.0,29.1,13913.839,76.88,28.0,3.0


TEST for mapping

In [5]:
import geopandas as gpd

In [6]:
my_access_token = 'pk.eyJ1IjoienV6aXRvbSIsImEiOiJjbHVwano4amkwNWtsMnFxcWs5cWQyNHd1In0.X97UO8zvIFBYdGhWJ4xXIQ'
px.set_mapbox_access_token(my_access_token)

In [7]:
map = px.scatter_mapbox(
        data_frame=df_dash_2,
        lat='latitude',
        lon='longitude',
        size='total_cases',
        size_max=20,
        color='total_cases',
        color_continuous_scale='OrRd',
        hover_data={'location': True},
        mapbox_style='dark',
        zoom=1,
        height=600,
        title=f'COVID-19 dashboard'
)

map.show()

CREATING DASHBOARD

In [8]:
# Define continents and metrics
continents = list(df_dash_2['continent'].unique())
metric_dict = {'total_cases': 'Total cases',
               'total_deaths': 'Total deaths',
               'total_tests': 'Total tests',
               'total_vaccinations': 'Total vaccinations',
               'people_fully_vaccinated': 'Number of fully vaccinated people'}

In [9]:
# Initialize the Dash app
app = dash.Dash()

# Define the layout of the app
app.layout = html.Div([
    html.H1('COVID-19 Tracker', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='continent-dropdown',
        options=[{'label': continent, 'value': continent} for continent in continents],
        value='Europe',
        multi=False,
        clearable=False,
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[{'label': label, 'value': metric} for metric, label in metric_dict.items()],
        value='total_cases',
        multi=False,
        clearable=False,
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(id='map-graph')
])


# Define callback to update the map based on dropdown values
@app.callback(
    Output('map-graph', 'figure'),
    [Input('continent-dropdown', 'value'),
     Input('metric-dropdown', 'value')]
)
def update_map(continent, metric):
    filtered_data = df_dash_2[df_dash_2['continent'] == continent]
    map_fig = px.scatter_mapbox(
        data_frame=filtered_data,
        lat='latitude',
        lon='longitude',
        size=metric,
        size_max=20,
        color=metric,
        color_continuous_scale='OrRd',
        hover_data={'location': True},
        mapbox_style='dark',
        zoom=1,
        height=600,
        title=f'COVID-19 - {metric.replace("_", " ").title()} in {continent}'
    )
    return map_fig

In [10]:
# Start the Dash app in a separate thread
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)


<IPython.core.display.Javascript object>